--- 
Microeconometrics | Summer 2021 | M.Sc. Economics, Bonn University 

# Replication of Angrist, J., and Evans, W. (1998). "Children and Their Parent's Labor Supply: Evidence from Exogenous Variation in Family Size". <a class="tocSkip">   

[Carolina Alvarez](https://github.com/carolinalvarez)
---

**Angrist, J.D., & Evans, W.N. (1998).** [Children and Their Parents' Labor Supply: Evidence from Exogenous Variation in Family Size](https://www.jstor.org/stable/116844?seq=1). *The American Economic Review*, 88(3). 450-477. 

# Table of contents
* [Introduction](#Introduction)
* [Identification Strategy](#Identification)
* [Empirical Methodology](#Empirical-Methodology)
* [Replication Angrist & Evans (1998)](#Replication-of-Angrist-&-Evans-(1998))
 * [Data & Descriptive Statistics](#Data-&-Descriptive-Statistics)

In [1]:
#%matplotlib inline
!pip install stargazer
import numpy as np
import pandas as pd
import pandas.io.formats.style
import seaborn as sns
import statsmodels as sm
import statsmodels.formula.api as smf
import statsmodels.api as sm_api
import matplotlib as plt
import copy
from IPython.display import HTML
from stargazer.stargazer import Stargazer

---
# Introduction 
---

Angrist and Evans (1998) study the causal mechanisms between fertility and the work effort of both men and women. The authors begin by explaining the theoretical and practical reasons of studying the relationship between fertility and labor supply. First, there has been development of economic models that link the family and the labor market. Second, the relationship between fertility and labor supply could explain the increase of women's participation in the labor market in the post-war period, where having fewer children could have increased the female labor-force share. Meanwhile, other studies have linked fertility with female withdraws from the labor market and lower wages compared to men.

The mayority of empirical studies related to childbearing and labor supply find a negative correlation between family size (i.e., fertility) and female labor force. However, in his assesment of Economics of the Family, Robert J. Willis argues that there has not been well-measured exogenous variables that allow to separate cause and effect relationships from correlations among variables such as delay of marriage, decline of childbearing, increase in divorces, and increase in female labor force participation.

In this vein, the authors argue that the problems concerning the causal association between family size and labor supply arises from the theoretical argument that both factors are jointly determinated. For example, some labor-supply econometric models often use child-status variables as regressors on hours of work. On the other hand, economic demographers usually measure the effect of wages on fertility. According to the authors, "*since fertility variables cannot be both dependent and exogenous at the same time, it seems unlikely that either sort of regression has a causal interpretation*". 

Angrist and Evans (1998) contribute by using an **instrumental variable strategy (IV)** based on the sex-mix of children in families with two or more kids. This captures parental preferences for mixed-sex siblings, where parents of same-sex children are much more likely to have an additional child.

**Endogeneity Problem**

<center>Fertility 🠊 Labor supply</center>
<center>Labor supply 🠊 Fertility </center>

**Instrument** 

<center>Dummy variable for whether the sex of the second child matches the sex of the first child</center> 




---
# Identification
--- 
![ERROR:Here should be causal graph 1](files/causal_graph_v1.png)


---
# Empirical Methodology
## Casual estimation with a Binary IV

\begin{equation}
Y = \alpha + \delta D + \epsilon
\end{equation}

\begin{equation}
E[Y] = E[\alpha + \delta D + \epsilon]= \alpha + \delta E[D] + E[\epsilon]
\end{equation}

We re-write it as a difference equation in Z and divide both sides by $ E[D|Z=1] - E[D|Z=0]$ which yields:

\begin{equation}
\frac{E[Y|Z=1]-E[Y|Z=0]}{E[D|Z=1]-E[D|Z=0]} =\frac{\delta (E[D|Z=1]-E[D|Z=0]) + (E[\epsilon|Z=1]-E[\epsilon|Z=0])}{E[D|Z=1]-E[D|Z=0]}
\end{equation}

If the data holds for the causal graph despicted above, then $Z$ has no association with $ /epsilon$ and therefore:

\begin{equation}
\frac{E[Y|Z=1]-E[Y|Z=0]}{E[D|Z=1]-E[D|Z=0]} =\delta
\end{equation}

Under these conditions, the ratio of the population-level association between Y and Z and between D and Z is equal to the causal effect of D on Y. Then, if $Z$ is associated with $D$ but not with $/upvarepsilon$, then the following is the IV-Estimator for infinite samples:

\begin{equation}
\hat{\delta}_{IV,WALD} = \frac{E_N[y_i|z_i=1] - E_N[y_i|z_i=0]}{E_N[d_i|z_i=1] - E_N[d_i|z_i=0]}
\end{equation}

This is the IV-Estimator, which is known as the Wald Estimator when the instrument is binary. The wald estimator takes the average difference in the observed outcome of those who were exposed to the instrumental variable and of those who were not. Then it takes the average difference between the ones in the treatment group who took the treatment and those in the tratment group who did not receive the treatment.

## IV Estimation as LATE Estimation

Imbends and Angrist (1994) developed a framework for classifiying individuals as: i) those who respond positively to an instrument; ii) those who remain unaffected by the instrument; iii) those who rebel against the instrument. When $D$ and $Z$ are binary variables, then they are four possible group of individuals:

| Status                                    |Potential treatment assignment         | 
| ------------------------------------------|:-------------------------------------:| 
| Compliers ($\tilde{C}=c$)                 | $D^{Z=0}=0; D^{Z=1}=1$                | 
| Defiers ($\tilde{C}=d$)                   | $D^{Z=0}=1 D^{Z=1}=0$                 | 
| Always takers ($\tilde{C}=a$)             | $D^{Z=0}=1 D^{Z=1}=1$                 |  
| Never takers ($\tilde{C}=n$)              | $D^{Z=0}=0 D^{Z=1}=0$                 |   


A valid instrument $Z$ for the casual effect of $D$ on $Y$ must satisfy three assumptions in order to identify the **LATE**:

* Independence assumption: ($Y^{1}, Y^{0}, D^{Z=1}, D^{Z=0} \indep Z$)

This is analogous to the assumption that $cov(Z, \varepsilon)=0$ in the traditinal IV litera
* Non-zero effect of instrument assumption: $k \neq 0$ for all $i$
* Monotonicity assumption: either $k \geq 0$ for all $i$ or $k \leq 0$ for all $i$ 


---

---
# Replication of Angrist & Evans (1998)
---

## Data & Descriptive Statistics

Angrist and Evans (1998) use two extracts from the Census Public Use Micro Samples(PUMS) that correspond to the year 1980 and 1990 respectively. The Census contains information on labor supply, the sex of mother's first two children, and an indicator of multiple births.

However, there is no retrospective fertility information in the PUMS data sets other than the total number of children ever born. That means, the census does not track children across households. The authors thus matched children to mothers within households accordingly to the following strategy: they attached people in a household labeled as *child* to a female householder or the spouse of a male householder. They deleted any mother for whom the number of children in the household did not match the total amount of children ever born. Also, in households with multiple families, relationship codes and subfamily identifiers were used to pair children with mothers.

The sample is then limited to mothers aged 21-35 whose oldest child was less than 18 years old at the time of the Census. There are two main reasons to restrict the data in such fashion. First, few women younger than age 21 have two children, thus taking into account younger women will decrease the number of observations for the instrumental variable *more than two children* . Second, a child over 18 is very likely to have moved to a different household. It is very unlikely that a woman aged 35 years old at the time of the census has a child 18 year old or more. Thus, restricting the sample to women aged 35 or less assures that the two children are still living in the household and therefore, still be financially dependent from their parents.

For the empirical analysis, the authors use two samples for each year of census. The first includes all women (after restricting the sample to mothers aged 21-35) with two or more children. The second sample includes only married women for testing the main theories of household production (e.g., Gronau, 1973) and exploring the impact of children as well on father's labor supply.

The following table summarizes the samples created by the authors and used for the empirical analysis:


| Year        | Sample        | Description                                                            |
| :----       | :----         |:----                                                                   |
| 1980        | Full sample   | Woman with two or more children, age 21-35 years old                   |
|             | Married sample| Couples married at time of census, only once and at time of first birth|                   
| 1990        | Full sample   | Woman with two or more children, age 21-35 years old                   |            
|             |Married sample | Woman married at time of census                                        |                                

Variables with information on timing of first marriage and the number of marriages is not available in the 1990 PUMS; thereore, for building the 1990 married sample, only the variable wheter the woman was married at the time of the census is considered.

In [2]:
census_1 = pd.read_stata("data/m_d_806_1.dta")
census_2 = pd.read_stata("data/m_d_806_2.dta")
data=census_1.append(census_2, ignore_index=False, verify_integrity=False, sort=False)
#data = prepare_data(data) from the auxiliary to be created after def variables

---
<span style="color:coral">**NOTE**:</span> The original data provided by the authors can be found [here](https://economics.mit.edu/faculty/angrist/data1/data/angev98). For this replication the data is split into two .dta-files due to size constraints.

---

In [3]:
data.describe()

,STATE,SEXK,AGEK,QTRBKID,RACEK,SPANISH,BIRTHPLK,SCHOOLK,GRADE,FINGRADE,...,CLASSD,WEEKSD,HOURSD,INCOME1D,INCOME2D,AWEEK79D,AHOUR79D,AINC1D,AINC2D,id
count,927267.000000,927267.000000,927267.000000,927267.000000,927267.000000,927267.000000,927267.000000,927267.000000,927267.000000,927267.000000,...,762843.000000,762843.000000,762843.000000,762843.000000,762843.000000,762843.000000,762843.000000,762843.000000,762843.000000,927267.000
mean,28.463958,0.488175,8.758242,2.522137,1.575727,0.148400,48.795570,0.956873,5.507927,0.825858,...,1.850998,47.399563,42.905771,16818.576214,1503.258842,0.200804,0.206107,0.311123,0.319551,463653.625
std,15.381372,0.499860,4.764916,1.113425,2.055951,0.610885,112.210375,0.685231,4.391520,0.454339,...,1.537924,11.307519,12.586895,11964.223274,7094.477677,0.740307,0.758842,0.827360,0.876777,267656.625
min,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,-9995.000000,0.000000,0.000000,0.000000,0.000000,1.000
25%,17.000000,0.000000,5.000000,2.000000,1.000000,0.000000,17.000000,1.000000,1.000000,1.000000,...,1.000000,50.000000,40.000000,9765.000000,0.000000,0.000000,0.000000,0.000000,0.000000,231817.500
50%,29.000000,0.000000,9.000000,3.000000,1.000000,0.000000,30.000000,1.000000,5.000000,1.000000,...,1.000000,52.000000,40.000000,16005.000000,0.000000,0.000000,0.000000,0.000000,0.000000,463634.000
75%,41.000000,1.000000,13.000000,4.000000,1.000000,0.000000,42.000000,1.000000,9.000000,1.000000,...,2.000000,52.000000,48.000000,22105.000000,0.000000,0.000000,0.000000,0.000000,0.000000,695450.500
max,56.000000,1.000000,17.000000,4.000000,13.000000,4.000000,997.000000,3.000000,22.000000,3.000000,...,7.000000,52.000000,99.000000,75000.000000,75000.000000,3.000000,3.000000,3.000000,3.000000,927267.000


In [4]:
data.head()

,STATE,SEXK,AGEK,QTRBKID,RACEK,SPANISH,BIRTHPLK,SCHOOLK,GRADE,FINGRADE,...,CLASSD,WEEKSD,HOURSD,INCOME1D,INCOME2D,AWEEK79D,AHOUR79D,AINC1D,AINC2D,id
0,1,1,9,3,1,0,1,1,6,1,...,1.0,52.0,40.0,28005.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,1,8,3,1,0,1,2,5,1,...,5.0,52.0,72.0,0.0,10005.0,0.0,0.0,0.0,3.0,2.0
2,1,0,9,1,1,0,1,1,5,1,...,5.0,16.0,48.0,0.0,16005.0,0.0,0.0,2.0,0.0,3.0
3,1,0,5,2,1,0,1,1,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
4,1,1,11,1,1,0,1,1,7,1,...,1.0,32.0,40.0,9925.0,0.0,0.0,0.0,3.0,3.0,5.0


In [3]:
#constructing same sex variables --> instrument
#boy 1rst
data["boy1st"] = np.NaN
data.loc[data.SEXK == 0, "boy1st"] = 1 #for boys
data.loc[data.SEXK == 1, "boy1st"] = 0 #for girls

#boy 2nd
data["boy2nd"] = np.NaN
data.loc[data.SEX2ND == 0, "boy2nd"] = 1 #for boys
data.loc[data.SEX2ND == 1, "boy2nd"] = 0 #for girls, some will have NAs because there is no second child

#gen var two boys
data["two_boys"] = np.where(
    (data["boy1st"] == 1) & (data["boy2nd"] ==1), 1, 0)
#data["two_boys"] = np.NaN
#data.loc[(data["boy1st"] == 1) & (data["boy2nd"] == 1), "two_boys"] = 1

#gen var two girls
data["two_girls"] = np.where(
    (data["boy1st"] == 0) & (data["boy2nd"] ==0), 1, 0)

#data["two_girls"] = np.NaN
#data.loc[(data["boy1st"] == 0) & (data["boy2nd"] == 0), "two_girls"] = 1

#gen var same sex
data["same_sex"] = np.where(
    ((data["two_boys"] == 1) | (data["two_girls"] ==1)), 1, 0)

#mixed sex
data["mixed_sex"] = np.where(
    ((data["boy1st"] == 1) & (data["boy2nd"] ==0)) | ((data["boy1st"] == 0) & (data["boy2nd"] ==1)) , 1, 0)


In [77]:
#constructing race indicators for mom and dad
#mom
data["blackm"] = np.where(
    (data["RACEM"]==2), 1, 0)
data["hispm"] = np.where(
    (data["RACEM"]==12), 1, 0)
data["whitem"] = np.where(
    (data["RACEM"]==1), 1, 0)
data["otheracem"] = np.where(
    ((data["RACEM"]!=1) & (data["hispm"] != 1) & (data["whitem"] != 1)), 1, 0)

#data["hispm"] = np.NaN
#data.loc[data.RACEM == 12, "hispm"] = 1
#data["whitem"] = np.NaN
#data.loc[data.RACEM == 1, "whitem"] = 1
#data["otheracem"] = np.NaN
#data.loc[(data["blackm"] != 1) & (data["hispm"] != 1) & (data["whitem"] != 1), "otheracem"] = 1 

#dad
data["blackd"] = np.NaN
data.loc[data.RACED== 2, "blackd"] = 1
data["hispd"] = np.NaN
data.loc[data.RACED == 12, "hispd"] = 1
data["whited"] = np.NaN
data.loc[data.RACED == 1, "whited"] = 1
data["otheraced"] = np.NaN
data.loc[(data["blackd"] != 1) & (data["hispd"] != 1) & (data["whited"] != 1), "otheraced"] = 1 


In [5]:
#constructing education var for mom
data["educm"] = np.where(
    (data["FINGRADM"] == 1) | (data['FINGRADM'] == 2), data["GRADEM"] - 3, data["GRADEM"] - 2) 

#highschool graduated
data["hsgrad"]= np.NaN
data.loc[data["educm"] == 12, "hsgrad"] = 1

#highschoool or more
data["moregrad"]= np.NaN
data.loc[data["educm"] > 12, "moregrad"] = 1

In [6]:
# constructing income variables for labor market supply
#dad income
data["total_incomed"]=(data.INCOME1D + np.maximum(data.INCOME2D, 0))*2.099173554 #taking into account neg values
#reported in INCOME2D and deflating wages as stated in Angrist and Evans (1998) 

#mom income
data["total_incomem"]=(data.INCOME1M + np.maximum(data.INCOME2M, 0))*2.099173554 #taking into account neg values
#reported in INCOME2M and deflating wages as stated in Angrist and Evans (1998) 

#family income
data["faminc"]=data.FAMINC*2.099173554
data["faminc_log"]=(np.log(np.maximum(data.faminc, 1)))

#non-mom income
data["nonmomi"]=data.faminc-(data.INCOME1M*2.099173554)
data["nonmomi_log"]=(np.log(np.maximum(data.nonmomi, 1)))

#mom worked last year
data["workedm"] = np.where(
    (data["WEEKSM"] > 0), 1, 0) 

#dad worked last year
data["workedd"] = np.where(
    (data["WEEKSD"] > 0), 1, 0)

In [7]:
#more than 1 children
data["more1k"] = np.where(
    (data["KIDCOUNT"] > 1), 1, 0) 

#more than 2 children
data["more2k"] = np.where(
    (data["KIDCOUNT"] > 2), 1, 0) 

#more than 3 children
data["more3k"] = np.where(
    (data["KIDCOUNT"] >= 3), 1, 0) 

In [8]:
#constructing age when first child was born

#year of birth dad
data["yobd"] = np.where(
    (data["QTRBTHD"] == 0), 80-data["AGED"], 79-data["AGED"]) 

data["ageqm"]=4*(80-data.YOBM)-data.QTRBTHM-1
data["ageqd"]=4*(80-data.yobd)-data.QTRBTHD
data["agefstm"]=(data.ageqm-data.AGEQK)/4 #age of mom when kid first born
data["agefstd"]=(data.ageqd-data.AGEQK)/4

In [11]:
#number of moms
data["moms_n"]=len(data)

In [78]:
#sample of moms aged between 21 and 35, second kid no old than 1 year
data_2=data[((data['AGEM']>=21) & (data['AGEM']<=35)) & (data['KIDCOUNT']>=2) & (data['AGEQ2ND']>4) & (data['agefstm']>=15) 
            & (data['ASEX']==0) & (data['AAGE']==0) & (data['AQTRBRTH']==0)  
            & (data['ASEX2ND']==0) & (data['AAGE2ND']==0) & (data['AQTRBRTH']==0)].copy()

data_2.index = range(len(data_2.index))

print("The sample of moms aged between 21 and 35 with second kid no older than 1 year old has", len(data_2), "observations.")

The sample of moms aged between 21 and 35 with second kid no older than 1 year old has 394840 observations.


In [13]:
data_2.describe()

,STATE,SEXK,AGEK,QTRBKID,RACEK,SPANISH,BIRTHPLK,SCHOOLK,GRADE,FINGRADE,...,workedd,more1k,more2k,more3k,yobd,ageqm,ageqd,agefstm,agefstd,moms_n
count,394840.000000,394840.000000,394840.000000,394840.000000,394840.000000,394840.000000,394840.000000,394840.000000,394840.000000,394840.000000,...,394840.000000,394840.0,394840.000000,394840.000000,333707.000000,394840.000000,333707.000000,394840.000000,333707.000000,394840.0
mean,28.674321,0.488912,9.112289,2.528148,1.599433,0.150160,47.596269,1.056030,5.601018,0.936270,...,0.822928,1.0,0.402064,0.402064,46.291232,120.015333,133.311938,20.515074,23.954106,927267.0
std,15.371484,0.499878,3.536165,1.111621,2.131755,0.598132,107.658081,0.555176,3.477270,0.331009,...,0.381730,0.0,0.490315,0.490315,4.984661,14.039271,19.918779,2.937905,4.472240,0.0
min,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,-2.000000,82.000000,60.000000,15.000000,2.250000,927267.0
25%,17.000000,0.000000,6.000000,2.000000,1.000000,0.000000,17.000000,1.000000,3.000000,1.000000,...,1.000000,1.0,0.000000,0.000000,43.000000,110.000000,121.000000,18.250000,21.000000,927267.0
50%,29.000000,0.000000,9.000000,3.000000,1.000000,0.000000,29.000000,1.000000,5.000000,1.000000,...,1.000000,1.0,0.000000,0.000000,46.000000,122.000000,133.000000,20.000000,23.500000,927267.0
75%,41.000000,1.000000,12.000000,4.000000,1.000000,0.000000,42.000000,1.000000,8.000000,1.000000,...,1.000000,1.0,1.000000,1.000000,49.000000,131.000000,145.000000,22.250000,26.250000,927267.0
max,56.000000,1.000000,17.000000,4.000000,13.000000,4.000000,996.000000,3.000000,16.000000,3.000000,...,1.000000,1.0,1.000000,1.000000,65.000000,141.000000,328.000000,33.750000,76.000000,927267.0


In [14]:
def table_sum_stats(data):
    """
    Creates Descriptive statistics.
    """
    variables = data[
        [
            "KIDCOUNT",
            "more2k",
            "boy1st",
            "boy2nd",
            "two_boys",
            "two_girls",
            "AGEM",
            "agefstm",
            "workedm",
            "WEEKSM",
            "HOURSM",
            "total_incomem",
            "faminc",
            "nonmomi",
            "same_sex"
        ]
    ]

    table = pd.DataFrame()
    table["Mean"] = variables.mean()
    table["Standard Deviation"] = variables.std()
    table = table.astype(float).round(2)
    table["Description"] = [
            "Children ever born",
            "More than two children",
            "First child was a boy",
            "Second child was a boy",
            "First two children were boys",
            "First two children were girls",
            "Age",
            "Age at first birth",
            "If worked for pay in year prior to the census",
            "Weeks worked in year prior to the census",
            "Average hours worked per week",
            "Labor earnings year prior to census, 1995 dollars",
            "Family income year prior to census, 1995 dollars",
            "Non-wife income",
            "aa"
        
    ]
    

    return table

In [15]:
table1=table_sum_stats(data_2)
table1

,Mean,Standard Deviation,Description
KIDCOUNT,2.55,0.81,Children ever born
more2k,0.40,0.49,More than two children
boy1st,0.51,0.50,First child was a boy
boy2nd,0.51,0.50,Second child was a boy
two_boys,0.26,0.44,First two children were boys
two_girls,0.24,0.43,First two children were girls
AGEM,30.12,3.51,Age
agefstm,20.52,2.94,Age at first birth
workedm,0.57,0.50,If worked for pay in year prior to the census
WEEKSM,20.83,22.29,Weeks worked in year prior to the census


In [10]:
# Creating the marriaged sample out of sample of moms aged between 21 and 35, second kid no old than 1 year (data_2)

#First creating illegit var

data_2["qtrmar"] = np.where((data_2["QTRMAR"] > 0), data_2["QTRMAR"] - 1, data_2["QTRMAR"])

data_2["yom"] = np.where((data_2["QTRBTHM"] <= data_2["qtrmar"]), data_2["YOBM"] + data_2["AGEMAR"], data_2["YOBM"] + data_2["AGEMAR"]+1)

    
data_2["dom_q"]=(data_2.yom + (data_2.qtrmar)/4)
data_2["do1b_q"]=(data_2.YOBK + (data_2.QTRBKID)/4)

data_2["illegit"]= np.NaN
data_2.loc[data_2["dom_q"] - data_2["do1b_q"] > 0, "illegit"] = 1
data_2.loc[data_2["dom_q"] - data_2["do1b_q"] <= 0, "illegit"] = 0

#creating the sample for married couples 

msample=data_2[(data_2['TIMESMAR']==1) & (data_2['MARITAL']==0) & (data_2['illegit']==0) & (data_2['agefstd']>=15) &
            (data_2['agefstm']>=15) & (data_2["AGED"]!=np.NaN)]

In [11]:
# Creating the marriaged sample out of total sample of moms

data["qtrmar"] = np.where((data["QTRMAR"] >= 0), data["QTRMAR"] - 1, data["QTRMAR"])

data["yom"] = np.where((data["QTRBTHM"] <= data["qtrmar"]), data["YOBM"] + data["AGEMAR"], data["YOBM"] + data["AGEMAR"]+1)

    
data["dom_q"]=(data.yom + (data.qtrmar)/4)
data["do1b_q"]=(data.YOBK + (data.QTRBKID)/4)

data["illegit"]= np.NaN
data.loc[data["dom_q"] - data["do1b_q"] > 0, "illegit"] = 1
data.loc[data["dom_q"] - data["do1b_q"] <= 0, "illegit"] = 0

#creating the sample for married couples 

msample_total=data[((data['AGEM']>=21) & (data['AGEM']<=35)) & (data['TIMESMAR']==1) & (data['MARITAL']==0) & (data['illegit']==0) & (data['agefstd']>=15) & 
            (data['agefstm']>=15) & (data["AGED"]!=np.NaN)]

In [18]:
print("The sample of married couples has", len(msample), "observations.")

The sample of married couples has 254652 observations.


In [19]:
msample.describe()

,STATE,SEXK,AGEK,QTRBKID,RACEK,SPANISH,BIRTHPLK,SCHOOLK,GRADE,FINGRADE,...,ageqm,ageqd,agefstm,agefstd,moms_n,qtrmar,yom,dom_q,do1b_q,illegit
count,254652.000000,254652.000000,254652.000000,254652.000000,254652.000000,254652.000000,254652.000000,254652.000000,254652.000000,254652.000000,...,254652.000000,254652.000000,254652.000000,254652.000000,254652.0,254652.000000,254652.000000,254652.000000,254652.000000,254652.0
mean,29.005722,0.485643,8.678008,2.537373,1.503016,0.136567,49.403845,1.077129,5.206533,0.921234,...,121.092868,133.633614,21.216437,24.351623,927267.0,1.566640,68.936564,69.328224,71.193220,0.0
std,15.330749,0.499795,3.457543,1.108052,2.051482,0.572837,111.008018,0.601241,3.408550,0.339413,...,13.545744,18.224051,2.903868,3.989071,0.0,1.031434,3.339836,3.331400,3.449816,0.0
min,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,82.000000,77.000000,15.000000,15.000000,927267.0,0.000000,57.000000,57.000000,62.500000,0.0
25%,17.000000,0.000000,6.000000,2.000000,1.000000,0.000000,17.000000,1.000000,2.000000,1.000000,...,112.000000,122.000000,19.000000,21.750000,927267.0,1.000000,67.000000,67.000000,68.750000,0.0
50%,29.000000,0.000000,8.000000,3.000000,1.000000,0.000000,31.000000,1.000000,5.000000,1.000000,...,123.000000,133.000000,21.000000,23.750000,927267.0,2.000000,69.000000,69.500000,71.500000,0.0
75%,42.000000,1.000000,11.000000,4.000000,1.000000,0.000000,42.000000,1.000000,8.000000,1.000000,...,132.000000,145.000000,23.250000,26.500000,927267.0,2.000000,71.000000,71.750000,74.000000,0.0
max,56.000000,1.000000,17.000000,4.000000,13.000000,4.000000,996.000000,3.000000,16.000000,3.000000,...,141.000000,328.000000,33.500000,76.000000,927267.0,3.000000,79.000000,79.000000,79.000000,0.0


In [16]:
table1_2=table_sum_stats(msample)
table1_2

,Mean,Standard Deviation,Description
KIDCOUNT,2.51,0.77,Children ever born
more2k,0.38,0.49,More than two children
boy1st,0.51,0.50,First child was a boy
boy2nd,0.51,0.50,Second child was a boy
two_boys,0.27,0.44,First two children were boys
two_girls,0.24,0.43,First two children were girls
AGEM,30.39,3.39,Age
agefstm,21.22,2.90,Age at first birth
workedm,0.53,0.50,If worked for pay in year prior to the census
WEEKSM,19.02,21.87,Weeks worked in year prior to the census


In [17]:
keys = ['All women, PUMS 1980', 'Married couples, PUMS 1980']
frames = [table1, table1_2]
table2 = pd.concat(frames, axis=1, keys=keys) 
table2 = table2.drop(table2.columns[[2]], axis=1) 
table2

All women, PUMS 1980                     \
                              Mean Standard Deviation   
KIDCOUNT                      2.55               0.81   
more2k                        0.40               0.49   
boy1st                        0.51               0.50   
boy2nd                        0.51               0.50   
two_boys                      0.26               0.44   
two_girls                     0.24               0.43   
AGEM                         30.12               3.51   
agefstm                      20.52               2.94   
workedm                       0.57               0.50   
WEEKSM                       20.83              22.29   
HOURSM                       18.80              18.92   
total_incomem              7160.82           10804.13   
faminc                    42342.34           26563.21   
nonmomi                   35417.87           25741.83   
same_sex                      0.51               0.50   

              Married couples, PUMS 1980                     \
                                    Mean Standard Deviation   
KIDCOUNT                            2.51               0.77   
more2k                              0.38               0.49   
boy1st                              0.51               0.50   
boy2nd                              0.51               0.50   
two_boys                            0.27               0.44   
two_girls                           0.24               0.43   
AGEM                               30.39               3.39   
agefstm                            21.22               2.90   
workedm                             0.53               0.50   
WEEKSM                             19.02              21.87   
HOURSM                             16.70              18.34   
total_incomem                    6250.33           10210.87   
faminc                          47645.36           25821.97   
nonmomi                         41635.85           24735.76   
same_sex                            0.51               0.50   

                                                                  
                                                     Description  
KIDCOUNT                                      Children ever born  
more2k                                    More than two children  
boy1st                                     First child was a boy  
boy2nd                                    Second child was a boy  
two_boys                            First two children were boys  
two_girls                          First two children were girls  
AGEM                                                         Age  
agefstm                                       Age at first birth  
workedm            If worked for pay in year prior to the census  
WEEKSM                  Weeks worked in year prior to the census  
HOURSM                             Average hours worked per week  
total_incomem  Labor earnings year prior to census, 1995 dollars  
faminc          Family income year prior to census, 1995 dollars  
nonmomi                                          Non-wife income  
same_sex                                                      aa

In [22]:
data["INCOME1D"].describe()
data["INCOME2D"].describe()
msample["qtrmar"].describe()
sum(pd.isnull(data['agefstd']))

164424

In [18]:
variables1=["(1) one girl", "(2) one boy", "difference (2) - (1)"]
frequencies1=['Fraction of sample', 'Fraction that had another child']

def Table_3_panel_1(data):
    Table_3_panel1 = pd.DataFrame(np.nan, index=variables1, columns=frequencies1)
    Table_3_panel1.index.name="Sex of first child in families with one or more children"
    a=pd.DataFrame(data['boy1st'].value_counts(normalize=True)*100)
    a1=a.iloc[(0,0)]
    b1=a.iloc[(1,0)]

    c=pd.DataFrame(data.groupby("boy1st")["more1k"].value_counts(normalize=True) * 100)
    c1=c.iloc[(0,0)]
    d1=c.iloc[(2,0)]
    diff1=d1-c1

    Table_3_panel1.iloc[0,0]=a1
    Table_3_panel1.iloc[1,0]=b1

    Table_3_panel1.iloc[0,1]=c1
    Table_3_panel1.iloc[1,1]=d1
    Table_3_panel1.iloc[2,1]=diff1

    return Table_3_panel1

In [19]:
variables=["(1) one girl one boy", "(2) two boys", "(3) two girls", "(4) both same sex", "Difference (4) - (1)"]
frequencies=['Fraction of sample', 'Fraction that had another child']

def Table_3_panel_2(data):
    
    Table_3_panel2 = pd.DataFrame(np.nan, index=variables, columns=frequencies)
    Table_3_panel2.index.name="Sex of first child in families with two or more children"
    a2=pd.DataFrame(data['mixed_sex'].value_counts(normalize=True)*100)
    a2=a2.iloc[(1,0)]
    b2=pd.DataFrame(data['two_boys'].value_counts(normalize=True)*100)
    b2=b2.iloc[(1,0)]
    c2=pd.DataFrame(data['two_girls'].value_counts(normalize=True)*100)
    c2=c2.iloc[(1,0)]
    d2=pd.DataFrame(data['same_sex'].value_counts(normalize=True)*100)
    d2=d2.iloc[(0,0)]

    f2=pd.DataFrame(data.groupby("mixed_sex")["more3k"].value_counts(normalize=True) * 100)
    f2=f2.iloc[(3,0)]
    g2=pd.DataFrame(data.groupby("two_boys")["more3k"].value_counts(normalize=True) * 100)
    g2=g2.iloc[(3,0)]
    h2=pd.DataFrame(data.groupby("two_girls")["more3k"].value_counts(normalize=True) * 100)
    h2=h2.iloc[(3,0)]
    i2=pd.DataFrame(data.groupby("same_sex")["more3k"].value_counts(normalize=True) * 100)
    i2=i2.iloc[(3,0)]
    diff2=(i2-f2)/100
    #sd_diff2=np.sum(np.square(diff2))

    Table_3_panel2.iloc[0,0]=a2
    Table_3_panel2.iloc[1,0]=b2
    Table_3_panel2.iloc[2,0]=c2
    Table_3_panel2.iloc[3,0]=d2
    Table_3_panel2.iloc[4,0]="-"
    #Table_3_panel2.iloc[5,0]="-"

    Table_3_panel2.iloc[0,1]=f2
    Table_3_panel2.iloc[1,1]=g2
    Table_3_panel2.iloc[2,1]=h2
    Table_3_panel2.iloc[3,1]=i2
    Table_3_panel2.iloc[4,1]=diff2
    #Table_3_panel2.iloc[5,1]=sd_diff2

    return Table_3_panel2


In [20]:
Table3_1_1=Table_3_panel_1(data)
Table3_1_2=Table_3_panel_1(msample_total)
Table3_2_1=Table_3_panel_2(data_2)
Table3_2_2=Table_3_panel_2(msample)

keys = ['All women, PUMS 1980', 'Married women, PUMS 1980']
frames1 = [Table3_1_1, Table3_1_2]
frames2 = [Table3_2_1, Table3_2_2]
table3_1 = pd.concat(frames1, axis=1, keys=keys) 
table3_2 = pd.concat(frames2, axis=1, keys=keys)

In [26]:
table3_1

All women, PUMS 1980  \
                                                     Fraction of sample   
Sex of first child in families with one or more...                        
(1) one girl                                                  51.182453   
(2) one boy                                                   48.817547   
difference (2) - (1)                                                NaN   

                                                                                    \
                                                   Fraction that had another child   
Sex of first child in families with one or more...                                   
(1) one girl                                                             70.614511   
(2) one boy                                                              70.695410   
difference (2) - (1)                                                      0.080899   

                                                   Married women, PUMS 1980  \
                                                         Fraction of sample   
Sex of first child in families with one or more...                            
(1) one girl                                                      51.494825   
(2) one boy                                                       48.505175   
difference (2) - (1)                                                    NaN   

                                                                                    
                                                   Fraction that had another child  
Sex of first child in families with one or more...                                  
(1) one girl                                                             71.398612  
(2) one boy                                                              71.351937  
difference (2) - (1)                                                     -0.046675

In [27]:
table3_2

All women, PUMS 1980  \
                                                     Fraction of sample   
Sex of first child in families with two or more...                        
(1) one girl one boy                                             49.461   
(2) two boys                                                    26.3719   
(3) two girls                                                    24.167   
(4) both same sex                                                50.539   
Difference (4) - (1)                                                  -   

                                                                                    \
                                                   Fraction that had another child   
Sex of first child in families with two or more...                                   
(1) one girl one boy                                                     37.197120   
(2) two boys                                                             42.268576   
(3) two girls                                                            44.115027   
(4) both same sex                                                        43.151522   
Difference (4) - (1)                                                      0.059544   

                                                   Married women, PUMS 1980  \
                                                         Fraction of sample   
Sex of first child in families with two or more...                            
(1) one girl one boy                                                49.4652   
(2) two boys                                                        26.6073   
(3) two girls                                                       23.9276   
(4) both same sex                                                   50.5348   
Difference (4) - (1)                                                      -   

                                                                                    
                                                   Fraction that had another child  
Sex of first child in families with two or more...                                  
(1) one girl one boy                                                     34.674193  
(2) two boys                                                             40.383140  
(3) two girls                                                            42.463730  
(4) both same sex                                                        41.368271  
Difference (4) - (1)                                                      0.066941

In [63]:
#TABLE 6
def OLS_Regressions_more2k(data1, data2):
    
    model1_table6=sm_api.OLS(data1["more2k"], sm_api.add_constant(data1["same_sex"])).fit()
    model2_table6=sm_api.OLS(data1["more2k"], sm_api.add_constant(data1[["same_sex", "boy1st", "boy2nd"]])).fit()
    model3_table6=sm_api.OLS(data1["more2k"], sm_api.add_constant(data1[["boy1st", "two_boys", "two_girls"]])).fit()
    model4_table6=sm_api.OLS(data2["more2k"], sm_api.add_constant(data2["same_sex"])).fit()
    model5_table6=sm_api.OLS(data2["more2k"], sm_api.add_constant(data2[["same_sex", "boy1st", "boy2nd"]])).fit()
    model6_table6=sm_api.OLS(data2["more2k"], sm_api.add_constant(data2[["boy1st", "two_boys", "two_girls"]])).fit()
    Table = Stargazer([model1_table6, model2_table6, model3_table6, model4_table6, model5_table6, model6_table6])
    Table.title("Table 6 Part 1: OLS Estimates of More than 2 children equations - PUMS 1980")
    Table.custom_columns(['All women', 'All women', 'All women', 'Married women', 'Married women', 'Married women'], 
                         [1, 1, 1, 1, 1, 1])
    Table.significant_digits(4)
    #Table.custom_columns('All women')
    
    return Table

In [64]:
OLS_Regressions_more2k(data_2, msample)

In [140]:
def OLS_Labor_Supply_Models(data, outcomes):

    table = pd.DataFrame(
        {
            "OLS (1)": [],
            "Std.err (1)": [],
        }
    )
    table["outcomes"] = outcomes
    table = table.set_index("outcomes")

    for outcome in outcomes:

        data=data_2
        model=sm_api.OLS(data[outcome], sm_api.add_constant(data[["more2k", 'AGEM', 'agefstm', "boy1st", "boy2nd", "blackm", "hispm", "otheracem"]]))
        result=model.fit()
        outputs = [
                result.params["more2k"],
                result.bse["more2k"],
        ]
        table.loc[outcome] = outputs
        table = table.round(3)

    return table

In [142]:
outcomes_labor_supply=["workedm", "WEEKSM", "HOURSM", "total_incomem"]

Table7=OLS_Labor_Supply_Models(data_2, outcomes_labor_supply)
Table7

,OLS (1),Std.err (1)
outcomes,,
workedm,-0.177,0.002
WEEKSM,-8.998,0.072
HOURSM,-6.667,0.062
total_incomem,-3768.197,35.391


* **Angrist, J., & Evans, W. (1998)**. *Children and Their Parents' Labor Supply: Evidence from Exogenous Variation in Family Size*, 88(3), The American Economic Review.




-------
Notebook by Carolina Alvarez | GitHub profile: https://github.com/carolinalvarez.

---